In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import os

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from env import user, password, host
#import acquire
#import prepare
#import explore

# Acquire Data <br>
###### Creating functions that will import data

In [3]:
# this function be used to access the SQL server
# the user, host, and password will come from importing 'env'

def get_connection(db, user=user, host=host, password=password):
    '''
    This function creates a connection to the Codeup db.
    It takes db argument as a string name.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [4]:
def new_telco_data():
    '''
    This function gets new telco data from the Codeup database.
    '''
    sql_query = """
                SELECT *
                FROM customers
                JOIN contract_types USING(contract_type_id)
                JOIN internet_service_types USING(internet_service_type_id)
                JOIN payment_types USING(payment_type_id);
                """
    # Read in dataframe from Codeuo
    df = pd.read_sql(sql_query,get_connection('telco_churn'))
    return df

In [5]:
def get_telco_data():
        '''
        This function gets telco data from csv, or otherwise from Codeup database.
        '''
        if os.path.isfile('telco.csv'):
            df = pd.read_csv('telco.csv', index_col = 0)
        else:
            df = new_telco_data()
            df.to_csv('telco.csv')
        return df

In [6]:
df = get_telco_data()
df.head

<bound method NDFrame.head of       payment_type_id  internet_service_type_id  contract_type_id customer_id  \
0                   2                         1                 3  0016-QLJIS   
1                   4                         1                 3  0017-DINOC   
2                   3                         1                 3  0019-GFNTW   
3                   4                         1                 3  0056-EPFBG   
4                   3                         1                 3  0078-XZMHT   
...               ...                       ...               ...         ...   
7038                2                         3                 1  9962-BFPDU   
7039                2                         3                 1  9967-ATRFS   
7040                4                         3                 1  9970-QBCDA   
7041                2                         3                 1  9975-SKRNR   
7042                3                         3                 1  9986-BONCE  

In [7]:
df = pd.DataFrame(df)
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [8]:
df.shape

(7043, 24)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
payment_type_id,7043.0,2.315633,1.148907,1.00,1.0,2.00,3.00,4.00
internet_service_type_id,7043.0,1.872923,0.737796,1.00,1.0,2.00,2.00,3.00
contract_type_id,7043.0,1.690473,0.833755,1.00,1.0,1.00,2.00,3.00
senior_citizen,7043.0,0.162147,0.368612,0.00,0.0,0.00,0.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
monthly_charges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


In [11]:
def object_vals(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            print(df[col].value_counts(dropna = False))

In [12]:
def col_range(df):
    stats_df = df.describe().T
    stats_df['range'] = stats_df['max'] - stats_df['min']
    return stats_df

In [13]:
# Create a function to summaris my dataframe

def summarize_df(df):
    print(f'The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.')
    print('------------------------------------------------------')
    print(df.info())
    print('------------------------------------------------------')
    print(df.head())
    print('------------------------------------------------------')
    object_vals(df)
    print('------------------------------------------------------')
    print(col_range(df))
    

In [14]:
summarize_df(df)

The dataframe has 7043 rows and 24 columns.
------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup      

# Prepare Data

In [15]:
# Check for null values
# No nulls found

df.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

In [34]:
def telco_split(df):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=123, 
                                            stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=123,
                                       stratify=train_validate.churn)
    return train, validate, test

In [65]:
def prep_telco(df):
    '''
    This function takes in df that was acquired from get_telco_data.
    Then it drops the SQL foreign keys, which are unnecessary:
    payment_type_id, internet_service_type_id, contract_type_id
    '''
    # Drop the unnecessary colums
    df = df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'])
    
    # Replace string values with numbers
    df.replace({'gender':{'male':1, 'female':0}}, inplace=True)
    df.replace({'partner':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'dependents':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'phone_service':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'multiple_lines':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'online_security':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'online_backup':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'device_protection':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'tech_support':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'streaming_tv':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'streaming_movies':{'Yes':1, 'No':0, 'No internet service':0}}, inplace=True)
    df.replace({'paperless_billing':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'churn':{'Yes':1, 'No':0}}, inplace=True)
    df.replace({'internet_service_type':{'None':0, 'DSL':1, 'Fiber optic':2}}, inplace=True)
    df.replace({'payment_type':{'Mailed check':0, 'Electronic check':1, 'Bank transfer (automatic)':2, 'Credit card (automatic)':3}}, inplace=True)
    
    # Split the dataset into train, validate, and test subsets
    train, validate, test = telco_split(df)
    
    return train, validate, test

In [48]:
# Split the dataset into train, validate, and test data subsets

train, validate, test = prep_telco(df)
print('The shape of the data subsets:')
print(f'train    -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test     -> {test.shape}')

The shape of the data subsets:
train    -> (3943, 20)
validate -> (1691, 20)
test     -> (1409, 20)


In [49]:
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
5642,Female,0,Yes,Yes,23,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,25.60,514.75,No,Two year,None,Credit card (automatic)
3338,Male,0,No,No,27,Yes,Yes,No,No,Yes,No,Yes,Yes,Yes,100.50,2673.45,No,One year,Fiber optic,Bank transfer (automatic)
4273,Female,1,No,No,7,Yes,No,No,No,No,No,Yes,Yes,Yes,89.15,574.35,No,Month-to-month,Fiber optic,Electronic check
2382,Female,0,Yes,No,47,Yes,No,No,Yes,No,No,Yes,No,Yes,58.60,2723.4,No,Month-to-month,DSL,Bank transfer (automatic)
2655,Male,0,No,No,41,Yes,No,No,Yes,No,Yes,Yes,Yes,Yes,99.65,4220.35,No,Two year,Fiber optic,Credit card (automatic)


# Explore Data

In [ ]:
def scatter_telco(df, x, y):
    for species, subset in df.groupby('species'):
        plt.scatter(subset[x], subset[y], label=species)
        plt.xlabel(x)
        plt.ylabel(y)
        plt.legend()

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [20]:
# create X & y version of train, where y is a series with just the target variable 
# and X are all the features except the target variable. 

#X_train = train.drop(columns=['churn'])
#y_train = train.churn

#X_validate = validate.drop(columns=['churn'])
#y_validate = validate.churn

#X_test = test.drop(columns=['churn'])
#y_test = test.churn

In [38]:
#X_train.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,contract_type,internet_service_type,payment_type
5642,4,3,3,1982-FEBTD,Female,0,Yes,Yes,23,Yes,...,No internet service,No internet service,No internet service,No internet service,Yes,25.60,514.75,Two year,None,Credit card (automatic)
3338,3,2,2,8982-NHAVY,Male,0,No,No,27,Yes,...,Yes,No,Yes,Yes,Yes,100.50,2673.45,One year,Fiber optic,Bank transfer (automatic)
4273,1,2,1,4115-NZRKS,Female,1,No,No,7,Yes,...,No,No,Yes,Yes,Yes,89.15,574.35,Month-to-month,Fiber optic,Electronic check
2382,3,1,1,9633-DENPU,Female,0,Yes,No,47,Yes,...,No,No,Yes,No,Yes,58.60,2723.4,Month-to-month,DSL,Bank transfer (automatic)
2655,4,2,3,5693-PIPCS,Male,0,No,No,41,Yes,...,No,Yes,Yes,Yes,Yes,99.65,4220.35,Two year,Fiber optic,Credit card (automatic)


In [39]:
#y_train.head()

5642    No
3338    No
4273    No
2382    No
2655    No
Name: churn, dtype: object